In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import xarray as xr


In [ ]:
ds_fpath = Path(".").resolve().parent / "derivatives" / "evoked" / "aep_evoked.nc"

ds = xr.open_dataset(ds_fpath)

channels = ["E7", "E106", "E13", "E6", "E112", "E31", "E80", "E37", "E55", "E87"]
df = ds.to_array().sel(channel=channels).mean("channel").squeeze().to_pandas()

In [ ]:
from scipy.stats import zscore
signal_means = df.abs().mean(axis=1)
threshold = zscore(signal_means) > 3
indices_to_drop = threshold[threshold].index # indices are subject ids
indices_to_drop

In [ ]:
df_long = df.drop(indices_to_drop).reset_index().melt(id_vars=["subject"], value_name="ERP")
df_long["ERP"] = df_long["ERP"] * 1e6 # Volts to Microvolts
df_long["time"] = df_long["time"] *  1000  # seconds to milliseconds
df_long_6 = df_long.loc[df_long["subject"].str.contains("ses-06")].copy()
df_long_6["age"] = 6
df_long_12 = df_long.loc[df_long["subject"].str.contains("ses-12")].copy()
df_long_12["age"] = 12
df_long_by_age = pd.concat([df_long_6, df_long_12], axis=0)

sns.set(style="darkgrid")
fig, ax = plt.subplots(constrained_layout=True)

colors_for_plot = sns.color_palette()[:2]
sns.lineplot(
    data=df_long_by_age,
    x="time",
    y="ERP",
    hue="age",
    palette=colors_for_plot,
    linewidth=0.5,
    ax=ax,
    )

ax.legend()
ax.set_title("Evokd Potentials to AEP, 6 and 12 Months")
ax.set_xlabel("Time (ms)")
ax.set_ylabel("ERP (uV)")
plt.show()
